##### Задание 1 

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>  

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

##### Задание 2
Функция из обязательной части задания должна быть расширена следующим образом:

- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;  
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:  
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>  

In [73]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [2]:
quieries = ['java', 'анализ данных']

In [80]:
#функция для формирования датафрейма с информацией о постах

def get_posts(query, page):
    habr_search = pd.DataFrame()
    url = 'https://habr.com/ru/search/page'    

    for p in range(page):   #перебираем страницы
        cur_url = url + str(p + 1) #ссылка на след страницу
        
        for r in query:  #перебираем запросы
            params = {'q':r} 
            res = requests.get(cur_url, params=params) #отправляем get запрос с параметрами
            time.sleep(0.3) #ставим задержку между запросами
            soup = BeautifulSoup(res.text) #формируем суп объект из полученных результатов
            articles = soup.find_all('article', class_="tm-articles-list__item") 

            for article in articles:
                class_h2 = 'tm-article-snippet__title tm-article-snippet__title_h2'
                class_likes = 'tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'
                try:
                    date = article.find('span', class_='tm-article-snippet__datetime-published').text
                    title = article.find('h2',class_=class_h2).text
                    link = f"https://habr.com{article.find('h2',class_=class_h2).find('a').get('href')}"
                    likes_text = article.find('span', class_=class_likes).get('title')
                    likes = re.search(r'\d',likes_text).group(0) 
                except:
                #помимо обычных постов иногда встречаются какие-то рекламные(megapost) статьи с другими тегами, нужно ли их так же обрабатывать?
                    pass

                row = {'date':date, 'title': title, 'link':link, 'likes': likes}
                habr_search = pd.concat([habr_search, pd.DataFrame([row])])
    
    return habr_search.reset_index(drop=True)

In [86]:
df_posts = get_posts(quieries, 6)

In [87]:
df_posts

,date,title,link,likes
0,вчера в 16:43,"Хранимые процедуры, функции и триггеры на Java",https://habr.com/ru/company/redsoft/blog/699732/,1
1,21 сентября в 09:23,Вышла версия Java SE 19,https://habr.com/ru/news/t/689394/,5
2,3 марта 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...,3
3,6 марта 2020 в 15:38,Что нового будет в Java 14,https://habr.com/ru/company/alconost/news/t/49...,1
4,20 октября 2020 в 14:21,«1С: Предприятие» будет работать на российской...,https://habr.com/ru/news/t/524284/,1
...,...,...,...,...
235,5 октября в 16:01,"Большим данным настал конец. Databricks, Snowf...",https://habr.com/ru/company/geekfactor/blog/68...,7
236,21 июня в 16:31,Яндекс Практикум запускает программу по матема...,https://habr.com/ru/company/yandex_praktikum/n...,1
237,6 августа 2019 в 14:14,DeepCode — система анализа кода на базе глубин...,https://habr.com/ru/company/itsumma/news/t/462...,1
238,6 августа 2021 в 19:53,Лингвистический анализ показал резкий рост деп...,https://habr.com/ru/news/t/571634/,2


In [88]:
# функция для добавления текста материала

def add_text(df):
    i=0
    for url in df['link']: #перебираем ссылки на посты из датафрейма  
        res = requests.get(url).text
        time.sleep(0.3)
        soup = BeautifulSoup(res)
        try:
            full_text = soup.find('div', class_='tm-article-body').text
        except:
            full_text = ''
        df.loc[i, 'text'] = full_text
        i+=1
    return df
    

In [89]:
add_text(df_posts)

,date,title,link,likes,text
0,вчера в 16:43,"Хранимые процедуры, функции и триггеры на Java",https://habr.com/ru/company/redsoft/blog/699732/,1,Всем привет! Сегодня мы расскажем о полезной ...
1,21 сентября в 09:23,Вышла версия Java SE 19,https://habr.com/ru/news/t/689394/,5,Oracle презентовала платформу Java SE 19 (Jav...
2,3 марта 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/company/itsumma/news/t/490...,3,"Согласно отчету RedMonk за январь 2020 года, ..."
3,6 марта 2020 в 15:38,Что нового будет в Java 14,https://habr.com/ru/company/alconost/news/t/49...,1,
4,20 октября 2020 в 14:21,«1С: Предприятие» будет работать на российской...,https://habr.com/ru/news/t/524284/,1,"\n\r\nКомпоненты системы «1С: Предприятие», к..."
...,...,...,...,...,...
235,5 октября в 16:01,"Большим данным настал конец. Databricks, Snowf...",https://habr.com/ru/company/geekfactor/blog/68...,7,Тридцать восемь миллиардов долларов — это не ...
236,21 июня в 16:31,Яндекс Практикум запускает программу по матема...,https://habr.com/ru/company/yandex_praktikum/n...,1,Яндекс Практикум разработал образовательную п...
237,6 августа 2019 в 14:14,DeepCode — система анализа кода на базе глубин...,https://habr.com/ru/company/itsumma/news/t/462...,1,Швейцарский стартап DeepCode разрабатывает си...
238,6 августа 2021 в 19:53,Лингвистический анализ показал резкий рост деп...,https://habr.com/ru/news/t/571634/,2,Исследователи Школы информатики и вычислитель...
